When you run a notebook cell that imports from the whendo packages, you need to set the NOTEBOOK FILE ROOT to \$\{workspaceFolder\} so  that imports from top-level packages work. For example, with the property's default value of \$\{fileDirName\}, the imports from dispatcher.etc... will fail since the notebook location is one directory down from top-level.





In [ ]:
from datetime import time, datetime, timedelta
from pydantic import BaseModel
import requests
import json
from itertools import product
from whendo.core.server import Server
from whendo.core.scheduler import Immediately
import whendo.core.schedulers.timed_scheduler as sched_x
import whendo.core.actions.file_action as file_x
import whendo.core.actions.list_action as list_x
import whendo.core.actions.sys_action as sys_x
import whendo.core.actions.dispatch_action as disp_x
from whendo.core.programs.simple_program import PBEProgram
from whendo.core.util import PP, TimeUnit, Dirs, DateTime, Now, DateTime2, Rez
from whendo.core.action import RezDict, Action, ActionRez
import whendo.core.resolver as resolver

def get(server:Server, path:str):
    response = requests.get(cmd(server, path))
    return handle_response(response)
def put(server:Server, path:str, data:BaseModel):
    response = requests.put(cmd(server, path), data.json())
    return handle_response(response)
def post(server:Server, path:str, data:BaseModel):
    response = requests.post(cmd(server, path), data.json())
    return handle_response(response)
def post_dict(server:Server, path:str, data:dict):
    response = requests.post(cmd(server, path), json.dumps(data))
    return handle_response(response)
def delete(server:Server, path:str):
    response = requests.delete(cmd(server, path))
    return handle_response(response)

def cmd(server:Server,path:str):
    return f"http://{server.host}:{server.port}{path}"

def handle_response(response):
    if response.ok:
        PP.pprint(response.json())
        return response.json()
    else:
        raise Exception(response.json())

home0 = Server(host='127.0.0.1', port = 8000, tags = {"server_name":["home0"], 'role': ['usual', 'sweet']})
home1 = Server(host='192.168.0.26', port = 8001, tags = {"server_name":["home1"], 'role': ['usual', 'sour']})
servers = [home0, home1]

In [ ]:
def inventory(server:Server):

    heart_1 = file_x.FileAppend(file="heartbeat1.txt", payload={'words':'heartbreak hotel'})
    post(server, '/actions/heartbeat1', heart_1)
    heart_2 = file_x.FileAppend(file="heartbeat2.txt", payload={'words':'nothing but heartaches'})
    post(server, '/actions/heartbeat2', heart_2)
    heart_3 = file_x.FileAppend(file="heartbeat3.txt", payload={'words':'heart of glass'})
    post(server, '/actions/heartbeat3', heart_3)

    append_1 = file_x.FileAppend(file="append_1.txt")
    post(server, '/actions/append_1', append_1)
    append_2 = file_x.FileAppend(file="append_2.txt")
    post(server, '/actions/append_2', append_2)
    append_3 = file_x.FileAppend(file="append_3.txt")
    post(server, '/actions/append_3', append_3)

    sys_info = sys_x.SysInfo()
    post(server, '/actions/sys_info', sys_info)
    pause = sys_x.Pause()
    post(server, '/actions/pause', pause)
    logic_1 = list_x.All(actions=[heart_1, heart_2])
    post(server, '/actions/logic1', logic_1)
    success = list_x.Success()
    post(server, '/actions/success', success)
    file_append = file_x.FileAppend(file="boomerang.txt")
    post(server, '/actions/file_append', file_append)
    mini_info = sys_x.MiniInfo()
    post(server, '/actions/mini_info', mini_info)
    scheduling_info = disp_x.SchedulingInfo()
    post(server, '/actions/scheduling_info', scheduling_info)
    dispatcher_dump = disp_x.DispatcherDump()
    post(server, '/actions/dispatcher_dump', dispatcher_dump)
    terminate = list_x.Terminate()
    post(server, '/actions/terminate', terminate)
    raise_if_0 = list_x.RaiseCmp(cmp=0, value=0)
    post(server, '/actions/raise_if_0', raise_if_0)
    integer = list_x.Result(value=1)

    info_append_1 = list_x.All(actions=[sys_info, mini_info, list_x.RezFmt(), append_1])
    info_append_2 = list_x.All(actions=[mini_info, list_x.RezFmt(), append_2])
    info_append_3 = list_x.All(actions=[dispatcher_dump, sys_info, list_x.RezFmt(), append_3])
    post(server, '/actions/info_append_1', info_append_1)
    post(server, '/actions/info_append_2', info_append_2)
    post(server, '/actions/info_append_3', info_append_3)

    raise_all_1 = list_x.All(actions=[list_x.Result(value=0), raise_if_0])
    raise_all_2= list_x.All(actions=[list_x.Result(value=1), raise_if_0])
    post(server, '/actions/raise_all_1', raise_all_1)
    post(server, '/actions/raise_all_2', raise_all_2)
    raise_uf_1 = list_x.UntilFailure(actions=[list_x.Result(value=0), raise_if_0])
    raise_uf_2= list_x.UntilFailure(actions=[list_x.Result(value=1), raise_if_0])
    post(server, '/actions/raise_uf_1', raise_uf_1)
    post(server, '/actions/raise_uf_2', raise_uf_2)
    raise_us_1 = list_x.UntilSuccess(actions=[list_x.Result(value=0), raise_if_0])
    raise_us_2 = list_x.UntilSuccess(actions=[list_x.Result(value=1), raise_if_0])
    post(server, '/actions/raise_us_1', raise_us_1)
    post(server, '/actions/raise_us_2', raise_us_2)

    format_1 = list_x.All(actions = [mini_info, sys_info, list_x.RezFmt()])
    post(server, '/actions/format_1', format_1)

    execute_action = disp_x.Exec(server_name="home", action_name="file_append")
    post(server, '/actions/execute_action', execute_action)
    execute_action_key_tags = disp_x.ExecKeyTags(key_tags={"role":["sour"]}, action_name="file_append")
    # execute_action_key_tags = disp_x.ExecKeyTags()
    post(server, '/actions/execute_action_key_tags', execute_action_key_tags)
    sys_info_key_tags = list_x.All(actions=[sys_info, execute_action_key_tags])
    post(server, '/actions/sys_info_key_tags', sys_info_key_tags)


    scheduler = sched_x.Randomly(time_unit=TimeUnit.second, low=5, high=10)
    post(server, '/schedulers/randomly_soon', scheduler)
    scheduler = sched_x.Timely(interval=1)
    post(server, '/schedulers/often', scheduler)
    morning, evening = time(6,0,0), time(18,0,0)
    scheduler = sched_x.Timely(interval=1, start=morning, stop=evening)
    post(server, '/schedulers/timely_day', scheduler)
    scheduler = sched_x.Timely(interval=1, start=evening, stop=morning)
    post(server, '/schedulers/timely_night', scheduler)
    scheduler = Immediately()
    post(server, '/schedulers/immediately', scheduler)

    program = PBEProgram().prologue("heartbeat1").epilogue("heartbeat3").body_element("often", "heartbeat2")
    post(server, '/programs/program1', program)
    info_append = PBEProgram().prologue("info_append_1").epilogue("info_append_3").body_element("often", "info_append_2")
    post(server, '/programs/info_append', info_append)



In [ ]:
def schedule(server:Server):
    if True:
        start = Now.dt()+timedelta(seconds=2)
        stop = start + timedelta(seconds=20)
        datetime2 = DateTime2(dt1=start, dt2=stop)
        post(server, "/programs/info_append/schedule", datetime2)
    elif True:
        start = Now.dt()
        stop = start + timedelta(seconds=20)
        datetime2 = DateTime2(dt1=start, dt2=stop)
        post(server, "/programs/program1/schedule", datetime2)
    elif True:
        get(server, '/schedulers/often/actions/logic1')
        dt = DateTime(date_time=Now.dt()+timedelta(seconds=10))
        post(server, '/schedulers/often/actions/logic1/expire', dt)
        post(server, '/schedulers/often/actions/heartbeat3/defer', dt)
    elif True: # write once to heartbeat1 & heartbeat2
        dt = DateTime(date_time=Now.dt()+timedelta(seconds=10))
        post(server, '/schedulers/immediately/actions/logic1/defer', dt)

In [ ]:
[get(server, '/dispatcher/clear') for server in servers]
for (serverA, serverB) in list(product(servers, servers)):
    post(serverA, f"/servers/{serverB.tags['server_name'][0]}", serverB)
[inventory(server) for server in servers]

In [ ]:
[schedule(server) for server in [home0, home1]]

In [ ]:
[get(server, '/dispatcher/load') for server in [home0, home1]]

In [ ]:
[get(server, '/actions/sys_info/execute') for server in [home0, home1]]

In [ ]:
[get(server, '/dispatcher/describe_all') for server in [home0, home1]]

In [ ]:
[get(server, '/actions/sys_info/execute') for server in [home0, home1]]

In [ ]:
[get(server, '/servers') for server in [home0, home1]]

In [ ]:
[get(server, '/jobs/run') for server in [home0, home1]]

In [ ]:
[get(server, '/jobs/stop') for server in [home0, home1]]

In [ ]:
[get(server, '/jobs/count') for server in [home0, home1]]

In [ ]:
get(home1, '/actions/sys_info_key_tags/execute')

In [ ]:
get(home1, '/actions/execute_action_key_tags/execute')

In [ ]:
execute_action = disp_x.Exec(server_name="home1", action_name="file_append")
execute_action_key_tags = disp_x.ExecKeyTags(key_tags={"role":["sour"]}, action_name="file_append") if False else disp_x.ExecKeyTags()
for svr in [home0,home1]:
    try:
        put(svr, '/actions/execute_action', execute_action)
    except:
        post(svr, '/actions/execute_action', execute_action)
    try:
        put(svr, '/actions/execute_action_key_tags', execute_action)
    except:
        post(svr, '/actions/execute_action_key_tags', execute_action)

In [ ]:
execute_action = disp_x.Exec(server_name="home0", action_name="file_append")
for svr in servers:
    try:
        put(svr, '/actions/execute_action', execute_action)
    except:
        post(svr, '/actions/execute_action', execute_action)
rez = Rez(result="Eureka!")
key_tags = {"role": ["sour"]}
rez_dict = RezDict(rez=rez, dictionary=key_tags)
post(home1, '/servers/by_tags/any/actions/execute_action/execute_with_rez',rez_dict)

In [ ]:
############## BWEE
############## BWEE
execute_action_key_tags = disp_x.ExecKeyTags(key_tags={"role":["sweet"]}, action_name="file_append")
for svr in servers:
    try:
        put(svr, '/actions/execute_action_key_tags', execute_action_key_tags)
    except:

        post(svr, '/actions/execute_action_key_tags', execute_action_key_tags)
rez = Rez(result="Eureka!")
key_tags = {"role": ["sweet"]} # execution location
rez_dict = RezDict(rez=rez, dictionary=key_tags)
post(home0, '/servers/by_tags/any/actions/execute_action_key_tags/execute_with_rez',rez_dict)

In [ ]:
test_action1 = disp_x.ExecKeyTags()
test_action2 = list_x.Vals(vals={"payload":"Guerneville!!!", "action_name":"file_append", "key_tags":{"role":["sweet"]}})
test_action3 = list_x.All(include_processing_info=True, actions=[test_action2,test_action1])
for svr in servers:
    try:
        put(svr, '/actions/test_action3', test_action3)
    except:
        post(svr, '/actions/test_action3', test_action3)
key_tags = {"role": ["sour"]} # top-level execution
post_dict(home0, '/servers/by_tags/any/actions/test_action3/execute', key_tags)

In [ ]:
test_action1 = disp_x.ExecKeyTags()
test_action_a = list_x.Result(value="Arcata!")
test_action_b = list_x.Vals(vals={"payload":{"x":"Eureka!!!"}})
test_action2 = list_x.Vals(vals={"action_name":"file_append", "key_tags":{"role":["sweet"]}})
test_action3 = list_x.All(include_processing_info=True, actions=[test_action_b, test_action2, test_action1])

for svr in servers:
    try:
        put(svr, '/actions/test_action3', test_action3)
    except:
        post(svr, '/actions/test_action3', test_action3)
key_tags = {"role": ["sour"]} # top-level execution
post_dict(home0, '/servers/by_tags/any/actions/test_action3/execute', key_tags)

In [ ]:
test_action1 = disp_x.ExecKeyTags()
test_action_a = list_x.Result(value="Arcata!")
test_action_b = list_x.Vals(vals={"payload":{"x":"Eureka!!!"}})
test_action2 = list_x.Vals(vals={"action_name":"file_append", "key_tags":{"role":["sweet"]}})
test_action3 = list_x.All(include_processing_info=True, actions=[test_action_b, test_action2, test_action_a, test_action1])
test_action3.json()
for svr in servers:
    try:
        put(svr, '/actions/test_action3', test_action3)
    except:
        post(svr, '/actions/test_action3', test_action3)
key_tags = {"role": ["sour"]} # top-level execution
post_dict(home0, '/servers/by_tags/any/actions/test_action3/execute', key_tags)

In [ ]:
"XXXXXXXXXXXXXXXX !!!!!!!!!"
file_append = resolver.resolve_action(get(home0, '/actions/file_append'))
test_action1 = disp_x.ExecSuppliedKeyTags(key_tags={"role":["sour"]})
test_action2 = list_x.Vals(vals={"action":file_append, "payload":{"x":"Petrolia!!!"}})
# test_action2 = list_x.Vals(vals={"action":file_append, "key_tags":{"role":["sour"]}, "payload":{"x":"Petrolia!!!"}})
test_action3 = list_x.All(include_processing_info=True, actions=[test_action2, test_action1])
for svr in servers:
    try:
        put(svr, '/actions/test_action3', test_action3)
    except:
        post(svr, '/actions/test_action3', test_action3)
key_tags = {"role": ["sweet"]} # top-level execution
post_dict(home1, '/servers/by_tags/any/actions/test_action3/execute', key_tags)

In [ ]:
"YYYYYYYYYYYYYYYYY !!!!!!!!!"

file_append = resolver.resolve_action(get(home0, '/actions/file_append'))
file_append.payload = {"x":"Petrolia!!!"}
test_action1 = disp_x.ExecSupplied() #KeyTags(key_tags={"role":["sour"]})
test_action2 = list_x.Vals(vals={"action":file_append})
test_action3 = list_x.All(include_processing_info=True, actions=[test_action2, test_action1])
for svr in servers:
    try:
        put(svr, '/actions/test_action3', test_action3)
    except:
        post(svr, '/actions/test_action3', test_action3)
get(home0, '/actions/test_action3/execute')

In [ ]:
"ZZZZZZZZZZZZ !!!!!!!!!"

file_append = resolver.resolve_action(get(home0, '/actions/file_append'))
file_append.payload = {"x":"Petrolia!!!"}
test_action1 = disp_x.ExecSupplied(action=file_append) #KeyTags(key_tags={"role":["sour"]})
# test_action2 = list_x.Vals(vals={"action":file_append})
test_action3 = list_x.All(include_processing_info=True, actions=[test_action1])
for svr in servers:
    try:
        put(svr, '/actions/test_action3', test_action3)
    except:
        post(svr, '/actions/test_action3', test_action3)
get(home0, '/actions/test_action3/execute')

In [ ]:
file_append = resolver.resolve_action(get(home0, '/actions/file_append'))
test_action1 = disp_x.ExecSuppliedKeyTags(action=file_append)
test_action_b = list_x.Vals(vals={"payload":{"x":"Petrolia!!!"}})
test_action2 = list_x.Vals(vals={"key_tags":{"role":["sour"]}})
test_action3 = list_x.All(include_processing_info=True, actions=[test_action_b, test_action2, test_action1])
for svr in servers:
    try:
        put(svr, '/actions/test_action3', test_action3)
    except:
        post(svr, '/actions/test_action3', test_action3)
key_tags = {"role": ["sweet"]} # top-level execution
post_dict(home1, '/servers/by_tags/any/actions/test_action3/execute', key_tags)

In [ ]:
from whendo.sdk.client import Client
cl = Client(host=home0.host, port=home0.port)
schedule_key_tags2 = disp_x.ExecSuppliedKeyTags(key_tags={"role":["sour"]}, action = disp_x.ScheduleProgram(program_name="info_append"))
cl.set_action(action_name="schedule_key_tags2", action=schedule_key_tags2)
rez=Rez(flds={"start_stop": DateTime2(dt1=Now.dt() + timedelta(seconds=3), dt2= Now.dt() + timedelta(seconds=45))})
cl.execute_action_with_rez(action_name="schedule_key_tags2", rez=rez)

In [ ]:
from whendo.core.resolver import resolve_action_rez
dd = {'dt1': datetime(2021, 5, 1, 2, 6, 40, 932219), 'dt2': datetime(2021, 5, 1, 2, 7, 22, 932301)}
resolve_action_rez(dictionary=dd)

In [ ]:
import whendo.core.util as util
file = "/Users/electronhead/.whendo/whendo/output/foo.txt"
with open(file, "a") as outfile:
    util.PP.pprint(False, stream=outfile)
    outfile.write("\n")

In [ ]:
from whendo.sdk.client import Client
action = file_x.FileAppend(file="test.txt")
action2 = list_x.All(actions=[list_x.Result(value=False), action])
Client(host=home0.host, port=home0.port).execute_supplied_action(supplied_action=action2)

In [ ]:
rez = resolver.resolve_rez(get(home0, '/actions/sys_info/execute'))
print (type(rez))

In [ ]:
from whendo.sdk.client import Client
Client(host=home0.host, port=home0.port).get_servers_by_tags(key_tags={"server_name":["home0"]}, mode="any")
# key_tags={"server_name":["home0"]}
# post_dict(home1, "/servers/by_tags/any", key_tags)

In [ ]:
from whendo.sdk.client import Client
PP.pprint(Client(host=home0.host, port=home0.port).load_dispatcher().dict())

In [ ]:
get(home0, '/actions/dispatcher_dump/execute')

In [ ]:
import logging
logger = logging.getLogger(__name__)
type(logger)